In [ ]:
from os import listdir
from os.path import isfile, join
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle

# from google.colab import drive
# drive.mount('/content/drive')
path_data = 'data/'

# Weather

In [ ]:
weather=pd.read_csv(path_data+'weather.csv', encoding='utf-8')
# 2019-9-12
weather=weather[(weather['일시']<'2019-09-12') | (weather['일시']>'2019-09-13')]
# weather.head()

idx=["일시","기온(°C)",
"강수량(mm)",
"풍속(m/s)","풍향(16방위)",
"습도(%)",
#"증기압(hPa)",
"이슬점온도(°C)",
"현지기압(hPa)",
#"해면기압(hPa)",
"일조(hr)",
#"전운량(10분위)",
#"중하층운량(10분위)",
"시정(10m)",
"지면온도(°C)"
]
weather=weather[idx]

In [ ]:
# 결측치
weather['일조(hr)'] = weather['일조(hr)'].fillna(0)
weather['강수량(mm)'] = weather['강수량(mm)'].fillna(0)
(weather.isnull().sum()/1128)*100

In [ ]:
#wind=pd.DataFrame({'풍향(sin)': np.sin(np.pi*weather["풍향(16방위)"]/360),'풍향(cos)': np.cos(np.pi*weather["풍향(16방위)"]/360)})
wind=pd.DataFrame({'풍향': np.cos(np.pi*weather["풍향(16방위)"]/360)})
weather=pd.concat((weather,wind),axis=1)
weather=weather.drop("풍향(16방위)",axis=1)
weather.head()

In [ ]:
# 결측치 처리
weather_c=weather.interpolate(method='linear')
print(weather_c.shape)
# 스케일링
data_w = weather_c.values[:,1:]
sc = MinMaxScaler(feature_range = (0, 1))
data_w = sc.fit_transform(data_w)
print(data_w.shape)
np.save(path_data+'data_w', data_w)

# 방위각

In [ ]:
fnames=[f for f in listdir(path_data+'sun/UR00000126') if isfile(join(path_data+'sun/UR00000126', f))]
fnames.sort()
#al
data_az=[]
for fn in fnames:
  tmp=pd.read_csv(path_data+'sun/UR00000126/'+fn).values[:,1]
  for i in range(24):
      tmpi=np.fromstring(tmp[i], dtype=float, sep=' ')
      tmp[i]=tmpi[0]+tmpi[1]/60.0+tmpi[2]/3600.0
      tmp[i]=180-tmp[i]
  data_az.append(tmp)
data_az=np.array(data_az).reshape((-1,1))
data_az=data_az.astype('float32')
#data_az = pd.DataFrame({'방위각_sin': np.sin(np.pi*data_az[:,0]/360),'방위각_cos': np.cos(np.pi*data_az[:,0]/360)})
data_az = pd.DataFrame({'방위각_cos': np.cos(np.pi*data_az[:,0]/360)})
data_az=np.array(data_az)
#al
data_al=[]
for fn in fnames:
  tmp=pd.read_csv(path_data+'sun/UR00000126/'+fn).values[:,2]
  for i in range(24):
      tmpi=np.fromstring(tmp[i], dtype=float, sep=' ')
      tmp[i]=tmpi[0]+tmpi[1]/60.0+tmpi[2]/3600.0
  data_al.append(tmp)
data_al=np.array(data_al).reshape((-1,1))#dataset1.flatten()[:,np.newaxis]#(1128, 1)
data_al=data_al.astype('float32')
#data_al = pd.DataFrame({'고도_sin': np.pi*np.sin(data_al[:,0]/90),'고도_cos': np.cos(np.pi*data_al[:,0]/90)})
data_al = pd.DataFrame({'고도_cos': np.cos(np.pi*data_al[:,0]/90)})
data_al=np.array(data_al)

sc = MinMaxScaler(feature_range = (0, 1))
data_al_scaled = sc.fit_transform(data_al)
sc = MinMaxScaler(feature_range = (0, 1))
data_az_scaled = sc.fit_transform(data_az)
data_sun = np.concatenate((data_al_scaled,data_az_scaled),axis=-1)
print(data_sun.shape)
np.save(path_data+'data_sun', data_sun)

# input/output

In [ ]:
n_steps_in=24*3
n_steps_out=24
n_step=n_steps_in


#weather, sun data
data_w = np.load(path_data + 'data_w.npy')
data_sun = np.load(path_data + 'data_sun.npy')
#pv data
data_hr1=[]
for fn in fnames:
    data_hr1.append(pd.read_csv(path_data + 'cv_26/20'+ fn ).values[:,0])
data_hr1=np.array(data_hr1)
data_hr1=np.concatenate((data_hr1,np.zeros((47,1))),axis=1)
print(data_hr1.shape,data_w.shape,data_sun.shape)

#faltten
data_hr1=data_hr1.reshape((-1,1))#dataset1.flatten()[:,np.newaxis]#(1128, 1)
clip=24*6
data_hr1=data_hr1[:-clip]
np.save(path_data+'data_hr1', data_hr1)
# Feature Scaling
sc1 = MinMaxScaler(feature_range = (0, 1))
data_hr_scaled1 = sc1.fit_transform(data_hr1)
data_hr_scaled1.shape,np.min(data_hr_scaled1),np.max(data_hr_scaled1)
with open(path_data + "sc.pickle","wb") as fw:
    pickle.dump(sc1, fw)

#merge
dataset_scaled1=np.concatenate((data_hr_scaled1,data_w[:-clip],data_sun[:-clip]),axis=-1)

n_features=dataset_scaled1.shape[1]
#train / test split
n_train=len(dataset_scaled1)-24*10
n_test=len(dataset_scaled1)-n_train
training_set_scaled1=dataset_scaled1[:n_train]
test_set_scaled1=dataset_scaled1[n_train:]
print(training_set_scaled1.shape,test_set_scaled1.shape)

In [ ]:
X_train = []
y_train = []
for i in range(n_step,22*24-24):
    X_train.append(training_set_scaled1[i-n_step:i])
    y_train.append(training_set_scaled1[i:i+24])
for i in range(22*24+n_step,n_train-24):
    X_train.append(training_set_scaled1[i-n_step:i])
    y_train.append(training_set_scaled1[i:i+24])
X_train, y_train = np.array(X_train), np.array(y_train)
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], n_features))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], n_features))
print("train",X_train.shape,y_train.shape)#(960, 168, 1),(960,)

X_test = []
y_test=[]
for i in range(n_step, n_test-24+1,24):
    X_test.append(test_set_scaled1[i-n_step:i])
    y_test.append(test_set_scaled1[i:i+24])
X_test,y_test = np.array(X_test),np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], n_features))
y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], n_features))
print("test",X_test.shape,y_test.shape)

np.save(path_data+'X_train', X_train)
np.save(path_data+'y_train', y_train)
np.save(path_data+'X_test', X_test)
np.save(path_data+'y_test', y_test)